In [10]:
import cProfile, io, pstats
import math
from pstats import SortKey
from time import time
import tempfile
import sys

import torch
from botorch.acquisition import PosteriorMean
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.optim import optimize_acqf

from botorch.test_functions import SixHumpCamel
from botorch.utils.transforms import normalize, standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

torch.set_default_dtype(torch.float64)
torch.manual_seed(123456)

start = time()

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=10)
print(f"Time to fit: {time() - start:.2f} seconds")

acqf = PosteriorMean(model)


NUM_RESTARTS = 1
RAW_SAMPLES = 1

with cProfile.Profile() as pr:
    for _ in range(100):
        optimize_acqf(
            acq_function=acqf,
            bounds=bounds,
            q=1,
            num_restarts=NUM_RESTARTS,
            raw_samples=RAW_SAMPLES,
        )

print(f"Generated 2: {time() - start:.2f} seconds")

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats("scipy")
None

Time to fit: 0.05 seconds
Generated 2: 1.50 seconds
         2542375 function calls (2341023 primitive calls) in 1.453 seconds

   Ordered by: cumulative time
   List reduced from 493 to 40 due to restriction <'scipy'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.001    0.000    1.351    0.014 /opt/anaconda3/lib/python3.9/site-packages/botorch/generation/gen.py:43(gen_candidates_scipy)
      100    0.001    0.000    1.274    0.013 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_minimize.py:45(minimize)
      100    0.009    0.000    1.271    0.013 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/lbfgsb.py:210(_minimize_lbfgsb)
     2492    0.009    0.000    1.231    0.000 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/optimize.py:65(_compute_if_needed)
     1358    0.001    0.000    1.228    0.001 /opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_differentiable_functions.py:231(_update_fun)
     1246    